**DSAA6100 Week3 - Ensemble Learning - Lab1**

<ul>
    <b>Content</b>:
    <li>AdaBoost</li>
    <li>XGBoost</li>
</ul>

In [121]:
import random
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from numpy.linalg import *
from sklearn.preprocessing import StandardScaler
np.random.seed(6100)

import base64

# **Importing the dataset**


We will be focusing on a dataset of Brazilian weather station observations.

In [122]:
df = pd.read_csv("data/observations.csv")
df = df.drop(["Unnamed: 0"], axis=1)
print(df.head())

   wsid                 mdct  prcp    stp   smax   smin  gbrd  temp  dewp  \
0   178  2007-11-06 00:00:00   NaN  982.5  982.5  981.3   NaN  29.3  12.1   
1   178  2007-11-06 01:00:00   NaN  983.2  983.2  982.5   NaN  29.0  13.5   
2   178  2007-11-06 02:00:00   NaN  983.5  983.5  983.2   NaN  27.4  14.0   
3   178  2007-11-06 03:00:00   NaN  983.7  983.7  983.4   NaN  25.8  16.9   
4   178  2007-11-06 04:00:00   NaN  983.7  983.8  983.6   NaN  25.4  16.4   

   tmax  dmax  tmin  dmin  hmdy  hmax  hmin  wdsp   wdct  gust  
0  29.7  16.8  25.5  10.8  35.0  58.0  32.0   3.2  101.0   6.5  
1  29.9  13.6  29.0  12.2  39.0  39.0  35.0   3.6   94.0   6.4  
2  29.0  14.0  27.4  13.6  44.0  44.0  39.0   2.5   93.0   6.9  
3  27.4  16.9  25.8  14.1  58.0  58.0  44.0   1.7   96.0   5.8  
4  26.3  17.0  25.3  16.4  57.0  58.0  56.0   3.1  110.0   7.5  


In [123]:
#Check that this outputs the data frame with 122000 rows and 19 columns
print(df.shape)

(122000, 19)


# **Data preprocessing**


First let us start by converting the "mdct" column from the weather dataset into a datetime. Look into to_datetime() function in pandas.

In [124]:
# STUDENT TODO: Convert the data type of the `mcdt` column in df to datetime
print("Original Datatype is "+str(df['mdct'].dtypes))
df['mdct'] = pd.to_datetime(df['mdct'])
print("Coverted Datatype is "+str(df['mdct'].dtypes))


Original Datatype is object
Coverted Datatype is datetime64[ns]


1.   Replace the missing values in the columns `gust, gbrd, wdsp, dewp, dmin, dmax` with 0.

2. Drop the rows where temp is 0.

3. Drop the column `prcp`.

In [125]:
# STUDENT TODO: Missing Processing
# Replace the missing values in the columns `gust, gbrd, wdsp, dewp, dmin, dmax` with 0.
df['gust'].fillna(0, inplace=True)
df['gbrd'].fillna(0, inplace=True)
df['wdsp'].fillna(0, inplace=True)
df['dewp'].fillna(0, inplace=True)
df['dmin'].fillna(0, inplace=True)
df['dmax'].fillna(0, inplace=True)
# Drop the rows where temp is 0
df = df[df['temp'] != 0]
# Drop the column `prcp`
df = df.drop('prcp', axis=1)

# Test Result
print('\n')
print(df.isnull().any())
print(df['temp'] == 0)



wsid    False
mdct    False
stp     False
smax    False
smin    False
gbrd    False
temp    False
dewp    False
tmax    False
dmax    False
tmin    False
dmin    False
hmdy    False
hmax    False
hmin    False
wdsp    False
wdct    False
gust    False
dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
121995    False
121996    False
121997    False
121998    False
121999    False
Name: temp, Length: 105200, dtype: bool


We want to calculate the difference in each metric over 1 hour. So basically if the temperature goes from 19.4 at 1200 hrs to 19.1 at 1300 hrs on a certain date we want the value to be -0.3.

Create two copies of the dataframe and shift the datetime by +1 hour on the right copy.

In [126]:
import datetime
# STUDENT TODO:
left_df = df.copy()
right_df = df.copy()

# Creat a timedelta used to represent 1 hour
delta = datetime.timedelta(hours=1)

# +1 hour on the right copy.
right_df['mdct'] = right_df['mdct'] + delta

print(df['mdct'])
print(right_df['mdct'])

0        2007-11-06 00:00:00
1        2007-11-06 01:00:00
2        2007-11-06 02:00:00
3        2007-11-06 03:00:00
4        2007-11-06 04:00:00
                 ...        
121995   2011-05-09 11:00:00
121996   2011-05-09 12:00:00
121997   2011-05-09 13:00:00
121998   2011-05-09 14:00:00
121999   2011-05-09 15:00:00
Name: mdct, Length: 105200, dtype: datetime64[ns]
0        2007-11-06 01:00:00
1        2007-11-06 02:00:00
2        2007-11-06 03:00:00
3        2007-11-06 04:00:00
4        2007-11-06 05:00:00
                 ...        
121995   2011-05-09 12:00:00
121996   2011-05-09 13:00:00
121997   2011-05-09 14:00:00
121998   2011-05-09 15:00:00
121999   2011-05-09 16:00:00
Name: mdct, Length: 105200, dtype: datetime64[ns]


1. Perform a left join between `left_df` and `right_df` on the `wsid` and `mcdt` columns. In the `merged_df` retain only the rows which have values from both `left_df` and `right_df`. (Clue: set `indicator` to be `True` while performing the merge and filter using the `_merge` column in the `merged_df`)

Understand how `merged_df` looks like using `.head(), .describe(), .info(), etc.` before proceeding further.

2. For the variables `stp, smax, smin, gbrd, dewp, tmax, dmax, tmin, dmin, hmdy, hmax, hmin, wdsp, wdct, gust, temp` (stored in the `columns` list), subtract the "_y" columns from their corresponding "_x" columns. For example, you should be subtracting `stp_y` column from the `stp_x` column in merged_df.

3. Store each of these subtraction results in a new column in `merged_df`. For example, store the result of `merged_df["stp_x"] - merged_df["stp_y"]` in `merged_df["stp"]`.

```
merged_df["stp"] = merged_df["stp_x"] - merged_df["stp_y"]
```

> Do this for all the variables in `columns`.

4. Drop the columns with "_x" and "_y" in their name from `merged_df`.

5. Drop the columns `['_merge', 'mdct', 'wsid']` from `merged_df`.

In [127]:
columns = ["stp", "smax", "smin", "gbrd", "dewp", "tmax",
            "dmax", "tmin", "dmin", "hmdy", "hmax",
            "hmin", "wdsp", "wdct", "gust", "temp"]


# STUDENT TODO:
# Merge two DataFrames, insert the 'wsid' column and the 'mcdt' column to the left, and use '_merge' column indicates the merge operation
print(left_df.head())# Print merged DataFrame
print(right_df.head())  # Print merged DataFrame
merged_df = left_df.merge(right_df[['wsid', 'mdct']], on=[
                          'wsid', 'mdct'], how='inner', indicator=True)
merged_df = merged_df[merged_df['_merge'] == 'both']#Keep only rows with values in both DataFrames
# merged_df.drop(columns=['_merge'], inplace=True)# Delete '_merge 'column
print(merged_df.head())# Print merged DataFrame


   wsid                mdct    stp   smax   smin  gbrd  temp  dewp  tmax  \
0   178 2007-11-06 00:00:00  982.5  982.5  981.3   0.0  29.3  12.1  29.7   
1   178 2007-11-06 01:00:00  983.2  983.2  982.5   0.0  29.0  13.5  29.9   
2   178 2007-11-06 02:00:00  983.5  983.5  983.2   0.0  27.4  14.0  29.0   
3   178 2007-11-06 03:00:00  983.7  983.7  983.4   0.0  25.8  16.9  27.4   
4   178 2007-11-06 04:00:00  983.7  983.8  983.6   0.0  25.4  16.4  26.3   

   dmax  tmin  dmin  hmdy  hmax  hmin  wdsp   wdct  gust  
0  16.8  25.5  10.8  35.0  58.0  32.0   3.2  101.0   6.5  
1  13.6  29.0  12.2  39.0  39.0  35.0   3.6   94.0   6.4  
2  14.0  27.4  13.6  44.0  44.0  39.0   2.5   93.0   6.9  
3  16.9  25.8  14.1  58.0  58.0  44.0   1.7   96.0   5.8  
4  17.0  25.3  16.4  57.0  58.0  56.0   3.1  110.0   7.5  
   wsid                mdct    stp   smax   smin  gbrd  temp  dewp  tmax  \
0   178 2007-11-06 01:00:00  982.5  982.5  981.3   0.0  29.3  12.1  29.7   
1   178 2007-11-06 02:00:00  983.2  9

We have created a copy of `merged_df` for you (`final_cleaned_df`).

1. Replace any non-negative `temp` value (`temp` >= 0) in `final_cleaned_df` with 1.
2. Replace any negative `temp` value (`temp` < 0) in `final_cleaned_df` with 0.

In [128]:
final_cleaned_df = merged_df.copy()
# STUDENT TODO:
...

Ellipsis

# **AdaBoost**

## **Logistic regression with sample weights**

As you will have learnt from the lectures, AdaBoost fits weak learners (here, logistic regression model)  in each iteration, to a dataset with weights $w_i$ attached to each sample $(x_i, y_i)$. The loss function now becomes:

> $
\mathcal{L}({\theta}) = -\sum_{i =1}^N w_{i} \times [ y_i\log(h_{{\theta}}({x}_i)) + (1 - y_i)\log(1 - h_{{\theta}}({x}_i))]
$

where $h_\theta(x)$ is the logistic regression hypothesis function.

The gradient of this weighted loss function with respect to the weight $\theta_j$ is given by:

> $\frac{\partial \mathcal{L}({\theta})}{\partial \theta_j} = \sum_{i=1}^{N}w_{i}(h_{{\theta}}({x}_i) - y_i)x_{ij}$

Using this information, complete the `compute_gradient` method in the `LogisticRegression` class to account for sample weights.

In [129]:
import numpy as np

In [130]:
class LogisticRegression:
    """
    Logistic Regression (aka logit, MaxEnt) classifier.

    Parameters
    ----------
    alpha: float, default=0.1
        Learning rate
    tol : float, default=0.01
        Tolerance for stopping criteria
    max_iter : int, default=1000
        Maximum number of iterations of gradient descent

    Attributes
    ----------
    theta_ : numpy.ndarray of shape (D + 1,)
        The value of the coefficients after gradient descent has converged
        or the number of iterations hit the maximum limit
    converged_: boolean
        Boolean value indicating whether gradient descent converged or not
    """

    def __init__(self, alpha=0.1, tol=0.01, max_iter=1000):

        self.alpha = alpha
        self.tol = tol
        self.max_iter = max_iter

        self.theta_ = None
        self.converged_ = False

    def compute_gradient(self, theta, X, y, sample_weight):
        """
        Compute the gradient of the cost function.

        Parameters
        ----------
        theta: numpy.ndarray of shape (D + 1,)
            The coefficients
        X: numpy.ndarray of shape (N, D + 1)
            The features matrix
        y: numpy.ndarray of shape (N,)
            The target variable array
        sample_weight: numpy.ndarray of shape (N,)
            The sample weight array

        Returns
        -------
        gradient: numpy.ndarray of shape (D + 1,)
            The gradient values
        """

        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        y_hat = sigmoid(X.dot(theta))

        # STUDENT TODO: Compute the gradient
        ...
        # STUDENT TODO END

    def fit(self, X, y, sample_weight):
        """
        Compute the coefficients using gradient descent and store them as theta_.

        Parameters
        ----------
        X: numpy.ndarray of shape (N, D)
            The features matrix
        y: numpy.ndarray of shape (N,)
            The target variable array
        sample_weight: numpy.ndarray of shape (N,)
            The sample weight array

        Returns
        -------
        Nothing
        """

        N, D = X.shape

        # Adding a column of ones at the beginning for the bias term
        ones_col = np.ones((N, 1))
        X = np.hstack((ones_col, X))

        # Initializing the weights
        theta_old = np.zeros((D + 1,))
        theta_new = theta_old.copy()

        for i in range(self.max_iter):
            theta_new = theta_old - self.alpha * self.compute_gradient(theta_old, X, y, sample_weight)

            if np.linalg.norm(theta_new - theta_old) / (np.linalg.norm(theta_old) + self.tol) <= self.tol:
                self.converged_ = True
                break

            theta_old = theta_new.copy()

        self.theta_ = theta_new

    def predict_proba(self, X):
        """
        Predict the probabilities that the data points in X belong to class 1.

        Parameters
        ----------
        X: numpy.ndarray of shape (N, D)
            The features matrix

        Returns
        -------
        y_hat: numpy.ndarray of shape (N,)
            The predicted probabilities that the data points in X belong to class 1
        """

        N = X.shape[0]

        # Adding a column of ones at the beginning for the bias term
        ones_col = np.ones((N, 1))
        X = np.hstack((ones_col, X))

        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        y_hat = sigmoid(X.dot(self.theta_))
        return y_hat

    def predict(self, X):
        """
        Predict the classes of the data points in X.

        Parameters
        ----------
        X: numpy.ndarray of shape (N, D)
            The features matrix

        Returns
        -------
        y_pred: numpy.ndarray of shape (N,)
            The predicted class of the data points in X
        """

        y_hat = self.predict_proba(X)
        y_pred = y_hat.copy()
        y_pred[y_pred >= 0.5] = 1
        y_pred[y_pred < 0.5] = 0
        return y_pred

### Test case for the `compute_gradient` method

In [131]:
def test_compute_gradient(StudentLogisticRegression):

    student_lr_clf = StudentLogisticRegression()
    np.random.seed(0)
    theta_tc = np.random.randn(2)
    X_tc = np.random.randn(100, 2)
    y_tc = np.random.randint(0, 2, 100)
    sample_weight_tc = np.random.uniform(0, 1, 100)
    student_ans = student_lr_clf.compute_gradient(theta_tc, X_tc, y_tc, sample_weight_tc)
    required_ans = np.array([12.903225675830651, -1.0829605960182223])

    assert np.linalg.norm(student_ans - required_ans) < 1e-2 * required_ans.size

test_compute_gradient(LogisticRegression)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

## **AdaBoostClassifier Implementation**

In this section, you will be implementing the AdaBoost classifier with the logistic regression weak learner from above.

### **Follow the hints in the `fit` method in the `AdaBoostClassifier` class to implement the following algorithm.**

Use the following Adaboost pseudocode as a reference.

**INPUT:**

1. training data $X, y = \{(x_{i}, y_{i})\}_{i=1}^N$

2. number of iterations $T$

**ALGORITHM:**

1.   Initialize $N$ uniform weights, i.e., $w_{1} = [1/N, 1/N, ..., 1/N]$

2.   `For` $t = 1, 2, ... T$:

> 2.1. Train model $h_t$ on $X$ and $y$ with instance weights $w_{t}$

> 2.2. Compute the weighted training error rate of $h_{t}$: $\epsilon_{t} = \sum_{i: y_i \ne h_t(x_i)} w_{t,i}$

> 2.3. If $\epsilon_{t} > 0.5$, flip $h_t$'s predictions

> 2.4. Set $\beta_{t} = \frac{1}{2}\text{ln}\left(\frac{1 - \epsilon_t}{\epsilon_t}\right)$

> 2.5. Update all instance weights: $w_{t + 1,i} = w_{t,i}\times\text{exp}(-\beta_{t}y_{i}h_{t}(x_{i}))$ $\forall i = 1, 2, ..., N$

> 2.6. Normalize $w_{t+1}$ such that the elements sum to 1

> `End For`

### **Follow the hints in the `predict` method in the `AdaBoostClassifier` class for obtaining the predictions of the trained AdaBoost classifier.**

> $H(x) = \text{sign}\left(\sum_{t=1}^{T}\beta_{t}h_{t}(x)\right)$

In [ ]:
class AdaBoostClassifier:
    """
    AdaBoost classifier based on logistic regression

    Parameters
    ----------
    T: int, default=100
        The number of logistic regression models in the boosting model

    Attributes
    ----------
    beta_arr_ : list of length T
        The list of beta values in the boosting model

    h_arr_: list of length T
        The list of logistic regression models in the boosting model
    """

    def __init__(self, T=100):

        self.T = T

        self.beta_arr_ = []
        self.h_arr_ = []

    def fit(self, X, y):
        """
        Train the logistic regression models (h) and compute their coefficients (beta),
        and store them in h_arr_ and beta_arr_ respectively.

        Parameters
        ----------
        X: numpy.ndarray of shape (N, D)
            The features matrix
        y: numpy.ndarray of shape (N,)
            The target variable array

        Returns
        -------
        Nothing
        """

        N = X.shape[0]

        # STUDENT TODO: Initialize w with appropriate values
        w = ...
        # STUDENT TODO END

        y_ = y.copy()
        # STUDENT TODO: Update y_ such that the 0's in y_ are replaced by -1
        ...
        # STUDENT TODO END

        for t in range(self.T):
            h = LogisticRegression()

            # STUDENT TODO: Fit h to X and y using w as the sample weights
            ...
            # STUDENT TODO END

            # STUDENT TODO: Obtain the predictions from h and compute epsilon
            y_pred = ...
            epsilon = ...
            # STUDENT TODO END

            # STUDENT TODO: If epsilon > 0.5:
            # 1. flip the predictions, i.e., replace 1's with 0's and 0's with 1's
            # 2. invert the model (h), i.e., make it predict 1 for what it predicted 0 earlier and vice-versa (clue: think about modifying h.theta_)
            ...
            # STUDENT TODO END

            self.h_arr_.append(h)

            if epsilon == 0:
                beta = np.inf
                self.beta_arr_.append(beta)
                break

            # STUDENT TODO: Compute beta
            beta = ...
            # STUDENT TODO END

            self.beta_arr_.append(beta)

            y_pred_ = y_pred.copy()

            # STUDENT TODO: Update y_pred_ such that the 0's in y_pred_ are replaced by -1
            ...
            # STUDENT TODO END

            # STUDENT TODO: Update w and normalize it such that the values in w sum to 1
            ...
            # STUDENT TODO END

    def predict(self, X):
        """
        Predict the classes of the data points in X.

        Parameters
        ----------
        X: numpy.ndarray of shape (N, D)
            The features matrix

        Returns
        -------
        y_pred: numpy.ndarray of shape (N,)
            The predicted class of the data points in X
        """

        N = X.shape[0]

        # Initialize the summation of beta times h for each x_i
        sum_beta_times_h = np.zeros((N,))

        for t in range(len(self.h_arr_)):

            # STUDENT TODO: Obtain the predictions of the t-th model in self.h_arr_
            # Replace the 0's in the array with -1
            ...
            # STUDENT TODO END

            # STUDENT TODO: Update sum_beta_times_h
            ...
            # STUDENT TODO END

        # STUDENT TODO: Create an array `y_pred` for the final predictions
        # Fill 0's and 1's in `y_pred` depending on the sum_beta_time_h value in the corresponding location
        ...
        # STUDENT TODO END


### Test case for the `fit` method

In [ ]:
def test_adaboost_fit(StudentAdaBoostClassifier):

    T = 4
    N = 100
    D = 2

    student_ab_clf = StudentAdaBoostClassifier(T=T)
    np.random.seed(0)
    X_tc = np.random.randn(N, D)
    y_tc = np.random.randint(0, 2, N)
    student_ab_clf.fit(X_tc, y_tc)

    beta_arr_student_ans = student_ab_clf.beta_arr_
    beta_arr_required_ans = np.array([0.08017132503758954, 0.046732864002838985,
                                      0.022808008179707476, 0.07012335626140642])
    assert np.linalg.norm(beta_arr_student_ans - beta_arr_required_ans) < 1e-2 * beta_arr_required_ans.size

    h_arr_student_ans = np.zeros([T, D + 1])

    for indx, h in enumerate(student_ab_clf.h_arr_):
        h_arr_student_ans[indx] = h.theta_

    h_arr_required_ans = np.array([[-0.01514967, -0.01713051,  0.21344566],
                                   [-0.01738886, -0.00656722,  0.12035635],
                                   [-0.0132557,  -0.00428943, 0.06616284],
                                   [-0.01037174, -0.00334141,  0.03943088]])

    assert np.linalg.norm(h_arr_student_ans - h_arr_required_ans) < 1e-2 * h_arr_required_ans.size

test_adaboost_fit(AdaBoostClassifier)

### Test case for the `predict` method

In [ ]:
def test_adaboost_predict(StudentAdaBoostClassifier):

    T = 4
    N = 100
    D = 2

    student_ab_clf = StudentAdaBoostClassifier(T=T)
    np.random.seed(0)
    X_tc = np.random.randn(N, D)
    y_tc = np.random.randint(0, 2, N)
    student_ab_clf.fit(X_tc, y_tc)

    student_ans = student_ab_clf.predict(X_tc)
    required_ans = [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
                    0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
                    1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
                    0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
                    1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]

    assert np.mean(student_ans == required_ans) >= 0.98

test_adaboost_predict(AdaBoostClassifier)

## **AdaBoost on the dataset**

Follow the hints in the `adaboost_on_dataset` method in the below cell to run `AdaBoostClassifier` on the dataset you prepared in section 1.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Splitting the final_cleaned_df into training and testing datasets
train_df, test_df = train_test_split(final_cleaned_df, test_size=0.2, random_state=6100)

def adaboost_on_dataset():
    """
    Trains the AdaBoostClassifier on a real-world dataset.

    Parameters
    ----------
    Nothing

    Returns
    -------
    y_test_pred: numpy.ndarray
        The predicted classes of the datapoints in test_df
    """

    # STUDENT TODO START: Initialize X_train and y_train with appropriate values (clue: use .iloc followed by .values of the DataFrame class)
    ...
    # STUDENT TODO END

    # STUDENT TODO START: Initialize X_test
    ...
    # STUDENT TODO END

    scaler = StandardScaler()
    # STUDENT TODO START: Scale the features of X_train and X_test using scaler
    ...
    # STUDENT TODO END

    clf = AdaBoostClassifier(T=10)
    # STUDENT TODO START: Now fit clf to the entire training data, i.e., X_train and y_train after feature scaling
    ...
    # STUDENT TODO END

    # STUDENT TODO START: Predict the classes of the datapoints in X_test and return the result
    ...
    # STUDENT TODO END
    return y_test_pred


# Calling the function
y_test_pred = adaboost_on_dataset()
# Calculate the accuracy
accuracy = accuracy_score(test_df['temp'], y_test_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# **XGBoost**

## TODOs for this section:
- You'll use xgboost library to build a classifier for the above problem. XGBoost is a popular library for gradient boosting, and you can find its documentation [here](https://xgboost.readthedocs.io/en/latest/).

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Splitting the final_cleaned_df into training and testing datasets
train_df, test_df = train_test_split(final_cleaned_df, test_size=0.2, random_state=6100)

# STUDENT TODO STARTS:
# 1. Fit an xgboost classifier to the training data (train_df, target variable is temp)
# 2. Obtain the predictions of the trained model on test_df in the variable y_test_pred
y_test_pred = ...
# STUDENT TODO ENDS

# Calculate the accuracy
accuracy = accuracy_score(test_df['temp'], y_test_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")